In [23]:
!pip install xgboost==2.0.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 43.0 MB/s  0:00:06m0:00:0100:01


In [2]:
!pip install feature_engine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 95.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [feature_engine]m [feature_engine]


In [31]:
import pandas as pd
import numpy as np
import os
import boto3
import pickle
import xgboost as xgb
import sklearn
import warnings
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, PowerTransformer, FunctionTransformer, OrdinalEncoder, StandardScaler
from feature_engine.encoding import RareLabelEncoder, MeanEncoder, CountFrequencyEncoder
from feature_engine.datetime import DatetimeFeatures
from feature_engine.outliers import Winsorizer
from feature_engine.selection import SelectBySingleFeaturePerformance

from sklearn.ensemble import RandomForestRegressor

import sagemaker
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
from sagemaker.tuner import (
    IntegerParameter,
    ContinuousParameter,
    HyperparameterTuner
)

In [4]:
pd.set_option('display.max_columns', None)
sklearn.set_config(transform_output="pandas")
warnings.filterwarnings('ignore')

In [5]:
train = pd.read_csv('train_data.csv')
train

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-03-21,Banglore,Delhi,08:55:00,19:10:00,615,1.0,In-flight meal not included,7832
1,Jet Airways,2019-03-27,Delhi,Cochin,17:30:00,04:25:00,655,1.0,In-flight meal not included,6540
2,Goair,2019-03-09,Banglore,Delhi,11:40:00,14:35:00,175,0.0,No info,7305
3,Air India,2019-06-12,Kolkata,Banglore,09:25:00,18:30:00,545,1.0,No info,8366
4,Jet Airways,2019-03-12,Banglore,Delhi,22:55:00,07:40:00,525,1.0,In-flight meal not included,11087
...,...,...,...,...,...,...,...,...,...,...
6690,Jet Airways,2019-03-21,Delhi,Cochin,10:45:00,18:50:00,1925,2.0,No info,11093
6691,Air India,2019-05-01,Kolkata,Banglore,09:25:00,18:30:00,545,1.0,No info,8891
6692,Jet Airways,2019-06-01,Delhi,Cochin,14:00:00,19:00:00,300,1.0,In-flight meal not included,10262
6693,Air Asia,2019-06-24,Delhi,Cochin,07:55:00,13:25:00,330,1.0,No info,6152


In [6]:
test = pd.read_csv('test_data.csv')
test

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-03-06,Banglore,Delhi,08:00:00,08:15:00,1455,1.0,No info,17996
1,Spicejet,2019-06-06,Kolkata,Banglore,22:20:00,00:40:00,140,0.0,No info,3873
2,Indigo,2019-03-18,Kolkata,Banglore,05:30:00,08:20:00,170,0.0,No info,4462
3,Indigo,2019-06-27,Chennai,Kolkata,19:35:00,21:55:00,140,0.0,No info,3597
4,Indigo,2019-05-06,Kolkata,Banglore,15:15:00,17:45:00,150,0.0,No info,4804
...,...,...,...,...,...,...,...,...,...,...
2088,Jet Airways,2019-05-27,Delhi,Cochin,19:15:00,12:35:00,1040,1.0,In-flight meal not included,12898
2089,Multiple Carriers,2019-06-27,Delhi,Cochin,11:25:00,19:15:00,470,1.0,No info,7155
2090,Jet Airways,2019-06-03,Delhi,Cochin,02:15:00,04:25:00,1570,1.0,In-flight meal not included,11627
2091,Multiple Carriers,2019-06-06,Delhi,Cochin,15:15:00,01:30:00,615,1.0,No info,6795


In [7]:
val = pd.read_csv('val_data.csv')
val

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-06-24,Delhi,Cochin,20:25:00,01:30:00,305,1.0,No info,5054
1,Multiple Carriers,2019-06-12,Delhi,Cochin,09:45:00,22:30:00,765,1.0,No info,9646
2,Jet Airways,2019-03-12,Banglore,Delhi,22:55:00,15:15:00,980,1.0,In-flight meal not included,11087
3,Multiple Carriers,2019-06-06,Delhi,Cochin,13:00:00,21:00:00,480,1.0,No info,13587
4,Jet Airways,2019-05-18,Delhi,Cochin,23:05:00,04:25:00,1760,2.0,No info,16704
...,...,...,...,...,...,...,...,...,...,...
1669,Spicejet,2019-05-01,Chennai,Kolkata,09:45:00,12:00:00,135,0.0,No info,3597
1670,Indigo,2019-05-01,Kolkata,Banglore,08:10:00,13:00:00,290,1.0,No info,5069
1671,Jet Airways,2019-05-27,Delhi,Cochin,05:30:00,12:35:00,425,2.0,In-flight meal not included,15544
1672,Jet Airways,2019-06-12,Mumbai,Hyderabad,19:35:00,21:05:00,90,0.0,In-flight meal not included,3210


## Preprocessor

In [8]:
airline_transformer = Pipeline(
steps=[
    ('simple_imputer', SimpleImputer(strategy='most_frequent')),
    ('group_rare_labels', RareLabelEncoder(tol=0.01,n_categories=3,replace_with='Other',missing_values='ignore')),
    ('one_hot_encoder',OneHotEncoder(sparse_output=False,handle_unknown='ignore'))
]
)

features_to_extract = ["month","week","day_of_week","day_of_year"]
date_of_journey_transformer = Pipeline(
    steps=[
        ('dt',DatetimeFeatures(features_to_extract=features_to_extract,yearfirst=True,format='mixed')),
        ('scaler',MinMaxScaler())
    ]
)

city_transformer = Pipeline(
    steps=[
        ('group_rare', RareLabelEncoder(tol=0.1, n_categories=3, replace_with='Other')),
        ('encoder', MeanEncoder()),
        ('scaler', PowerTransformer(standardize=True))
    ]
)
def is_north(X):
    north_cities = ['Delhi','Kolkata','Mumbai']
    cols = X.columns.to_list()
    return (X
        .assign(
        **{
            f"{col}_is_north": X.loc[:,col].isin(north_cities).astype('int') for col in cols
        }
        )
        .drop(columns=cols)
    )

location_transformer = FeatureUnion(transformer_list=[
    ("step_1",city_transformer),
    ("step_2",FunctionTransformer(func=is_north))
])

time_pipe_1 = Pipeline(
    steps=[
        ('time_feat',DatetimeFeatures(features_to_extract=['hour','minute'])),
        ('scaler',MinMaxScaler())
    ]
)

def get_part_of_day(X, early_morning=4, morning=8, noon=12, afternoon=16, evening=19, night=22, late_night=1):
    cols = X.columns.to_list()
    temp_time_df = X.assign(**{
        col: pd.to_datetime(X.loc[:,col]).dt.hour for col in cols
    })
    return temp_time_df.assign(**{
        col: np.select(
            [
                temp_time_df.loc[:,col].between(early_morning,morning,inclusive='left'),
                temp_time_df.loc[:,col].between(morning,noon,inclusive='left'),
                temp_time_df.loc[:,col].between(noon,afternoon,inclusive='left'),
                temp_time_df.loc[:,col].between(afternoon,evening,inclusive='left'),
                temp_time_df.loc[:,col].between(evening,night,inclusive='left'),
                temp_time_df.loc[:,col].between(night,late_night,inclusive='left'),
            ],
            ['early_morning','morning','noon','afternoon','evening','night'],
            default='late_night'
        ) for col in cols
    })

time_pipe_2 = Pipeline(steps=[
    ("part_of_the_day_trans", FunctionTransformer(func=get_part_of_day)),
    ("count_enco", CountFrequencyEncoder()),
    ("scaler", MinMaxScaler())
])

time_transformer = FeatureUnion([
    ('step_1', time_pipe_1,),
    ('step_2',time_pipe_2)
])

def duration_category(X, quick=0, medium=120, long=500):
    return (X.assign(duration_cat=np.select(
        [X.duration.between(quick,medium,inclusive='left'), 
         X.duration.between(medium, long, inclusive='left')],
        ['quick','medium'],
        default='long'
        )).drop(columns="duration"))


duration_pipe_1 = Pipeline(
    steps=[
        ("dur_cat",FunctionTransformer(func=duration_category)),
        ("ecoder", OrdinalEncoder(categories=[["quick", "medium", "long"]]))
    ]
)
duration_pipe_2 = Pipeline(steps=[
    ('outliers',Winsorizer(capping_method='iqr',fold=1.5)),
    ('impute', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

duration_trans = FeatureUnion([
    ("duration_pipe1",duration_pipe_2),
    ("duration_pipe2",duration_pipe_1)
])

def is_direct(X):
    return X.assign(direct_flight=X.total_stops.eq(0)).astype(int)

total_stops_trans = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('direct_check', FunctionTransformer(func=is_direct))
])

add_info_trans = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy='most_frequent')),
    ("rare_label", RareLabelEncoder(tol=0.1, n_categories=3, replace_with='other')),
    ("encoder", OneHotEncoder(sparse_output=False))
])

In [9]:
col_transformer = ColumnTransformer(
    [
        ('airline_trans',airline_transformer,['airline']),
        ('doj_trans',date_of_journey_transformer,['date_of_journey']),
        ('loc_trans',location_transformer,['source','destination']),
        ('time_trans',time_transformer,['dep_time','arrival_time']),
        ('dur_trans',duration_trans,['duration']),
        ('total_stops_trans', total_stops_trans,['total_stops']),
        ('add_info_trans', add_info_trans,['additional_info'])
    ],
    remainder='passthrough'
)

In [10]:
estimator = RandomForestRegressor(n_estimators=10, max_depth=3, random_state=42)

selection_trans = SelectBySingleFeaturePerformance(estimator, scoring='r2', threshold=0.1)

In [11]:
preprocessor = Pipeline(steps=[
    ("columns_transform", col_transformer),
    ("feature_selction", selection_trans)
])

In [12]:
preprocessor.fit(
    train.drop(columns="price"),
    train.price.copy()
)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('columns_transform', ...), ('feature_selction', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('airline_trans', ...), ('doj_trans', ...), ...]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'passthrough'
,"sparse_threshold sparse_threshold: float, default=0.3If

In [13]:
preprocessor.transform(train.drop(columns="price"))

,airline_trans__airline_Indigo,airline_trans__airline_Jet Airways,doj_trans__date_of_journey_week,doj_trans__date_of_journey_day_of_year,loc_trans__source,loc_trans__destination,dur_trans__duration,dur_trans__duration_cat,total_stops_trans__total_stops,total_stops_trans__direct_flight
0,0.0,1.0,0.176471,0.169492,-0.857930,-0.857930,-0.033916,2.0,1,0
1,0.0,1.0,0.235294,0.220339,1.065418,1.065418,0.046422,2.0,1,0
2,0.0,0.0,0.058824,0.067797,-0.857930,-0.857930,-0.917631,1.0,0,1
3,0.0,0.0,0.882353,0.872881,-0.203927,-0.203927,-0.174507,2.0,1,0
4,0.0,1.0,0.117647,0.093220,-0.857930,-0.857930,-0.214676,2.0,1,0
...,...,...,...,...,...,...,...,...,...,...
6690,0.0,1.0,0.176471,0.169492,1.065418,1.065418,2.597145,2.0,2,0
6691,0.0,0.0,0.529412,0.516949,-0.203927,-0.203927,-0.174507,2.0,1,0
6692,0.0,1.0,0.764706,0.779661,1.065418,1.065418,-0.666576,1.0,1,0
6693,0.0,0.0,1.000000,0.974576,1.065418,1.065418,-0.606322,1.0,1,0


## Upload to S3

In [26]:
BUCKET_NAME = 'flight-price-predictor-aws'
HOME = 'data'

In [16]:
def get_file_name(name):
    return f"{name}-pre.csv"

In [17]:
def export_data(data, name, pre):
    # split data into X and y subsets
    X = data.drop(columns="price")
    y = data.price.copy()
    
    # transformation
    X_pre = pre.transform(X)
    
    # exporting
    file_name = get_file_name(name)
    (
        y
        .to_frame()
        .join(X_pre)
        .to_csv(file_name, index=False, header=False)
    )

In [27]:
def upload_to_bucket(name):
    file_name = get_file_name(name)
    
    (
        boto3
        .Session()
        .resource("s3")
        .Bucket(BUCKET_NAME)
        .Object(os.path.join(HOME, f"{name}/{name}.csv"))
        .upload_file(file_name)
    )

In [19]:
def export_and_upload_bucket(data, name, pre):
    export_data(data, name, pre)
    upload_to_bucket(name)

In [28]:
export_and_upload_bucket(train, "train", preprocessor)
export_and_upload_bucket(test, "test", preprocessor)
export_and_upload_bucket(val, "val", preprocessor)

## Hyperparameter Tuning

In [32]:
session = sagemaker.Session()
region_name = session.boto_region_name

In [33]:
output_path = f"s3://{BUCKET_NAME}/model/output"

In [34]:
model = Estimator(
    image_uri=sagemaker.image_uris.retrieve("xgboost", region_name, "1.2-1"),
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type="ml.m4.xlarge",
    volume_size=5,
    output_path=output_path,
    use_spot_instances=True,
    max_run=300,
    max_wait=600,
    sagemaker_session=session
)


In [35]:
model.set_hyperparameters(
    objective="reg:linear",
    num_round=10,
    eta=0.1,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    alpha=0.1
)

In [36]:
hyperparameter_ranges = {
    "eta": ContinuousParameter(0.05, 0.2),
    "alpha": ContinuousParameter(0, 1),
    "max_depth": IntegerParameter(3, 5)
}

In [37]:
tuner = HyperparameterTuner(
    estimator=model,
    objective_metric_name="validation:rmse",
    hyperparameter_ranges=hyperparameter_ranges,
    strategy="Bayesian",
    objective_type="Minimize"
)

## Data Channels

In [38]:
def get_data_channel(name):
    bucket_path = f"s3://{BUCKET_NAME}/{HOME}/{name}"
    return TrainingInput(bucket_path, content_type="csv")

In [39]:
train_data_channel = get_data_channel("train")
train_data_channel

In [40]:
val_data_channel = get_data_channel("val")

In [41]:
data_channels = {
    "train": train_data_channel,
    "validation": val_data_channel
}

## Train & Tune Model

In [42]:
tuner.fit(data_channels)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


.........................................!


## Model Evaluation

In [43]:
with open("xgboost-model", "rb") as f:
    best_model = pickle.load(f)
    
best_model

In [44]:
def evaluate_model(name):
    file_name = get_file_name(name)
    data = pd.read_csv(file_name)
    
    X = xgb.DMatrix(data.iloc[:, 1:])
    y = data.iloc[:, 0].copy()
    
    pred = best_model.predict(X)
    
    return r2_score(y, pred)

In [45]:
evaluate_model("train")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 evaluate_model("train")                                                                      │
│   2                                                                                              │
│                                                                                                  │
│ in evaluate_model:10                                                                             │
│                                                                                                  │
│    7 │                                                                                           │
│    8 │   pred = best_model.predict(X)                                                            │
│    9 │                                                                                           │
│ ❱ 10 │   return r2_score(y, pred)                                                                │
│   11                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'r2_score' is not defined

In [ ]:
evaluate_model("train")